In [2]:
import os
from mapboxgl.viz import *
from mapboxgl.utils import *
import pandas as pd

token = os.getenv('MAPBOX_ACCESS_TOKEN')

stops = [
    ['reservoir', 'rgb(211,47,47)'], ['river', 'rgb(81,45,168)'], ['snow', 'rgb(2,136,209)'], 
    ['precip', 'rgb(139,195,74)'],  ['temp', 'rgb(255,160,0)']]
cdec = CircleViz('../data/cdec.geojson', radius=6, color_stops=stops, color_property='Gage Type', 
                 color_default='#10ac84', opacity=0.5)
healthcare = CircleViz('../data/healthcare_points.geojson', radius = 4, color_default='black')


data_url = 'https://raw.githubusercontent.com/mapbox/mapboxgl-jupyter/master/examples/data/points.csv'
df = pd.read_csv(data_url).round(3)

measure = 'Avg Medicare Payments'
heatmap_color_stops = create_color_stops([0.01, 0.25, 0.5, 0.75, 1], colors='Spectral')
heatmap_radius_stops = [[0, 3], [14, 100]] #increase radius with zoom

color_breaks = [round(df[measure].quantile(q=x*0.1), 2) for x in range(2,10)]
color_stops = create_color_stops(color_breaks, colors='Spectral')

heatmap_weight_stops = create_weight_stops(color_breaks)


heatmap = HeatmapViz('../data/healthcare_points.geojson', 
                     access_token=token,
                     weight_property = "Avg Medicare Payments",
                     weight_stops = heatmap_weight_stops,
                     color_stops = heatmap_color_stops,
                     radius_stops = heatmap_radius_stops,
                     opacity = 0.8,
                     center = (-95, 40),
                     zoom = 3,
                     below_layer='waterway-label'
                    )
lines = LinestringViz([], vector_url='mapbox://mapbox.mapbox-terrain-v2', 
                      vector_layer_name='contour', color_default='#ff6b6b', 
                      zoom=10, center=(-122.48, 37.83 ))

# make viz collection from list of MapViz objects
collection = VizCollection(viz_list=[cdec, healthcare, heatmap, lines], 
                           height='400px', center=(-122.48, 37.83), zoom=10)
collection.show()